In [1]:
import matplotlib.pyplot as plt
import numpy as np
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from keras.models import Sequential, load_model
import mysql.connector
from keras.optimizers import Adam
from sklearn import preprocessing
import datetime
cnx = mysql.connector.connect(user='root', password='941012',
                              host='127.0.0.1',
                              database='test0')
cursor = cnx.cursor()
query = ("SELECT n2,P1,T1,P2,T2 FROM test0.2016stablepoint LIMIT 1,25000")
cursor.execute(query)
result=cursor.fetchall()
x_list = np.array(result)
scaler = preprocessing.StandardScaler().fit(x_list)
train_x = scaler.transform(x_list)
#print train_x
query = ("SELECT P34 FROM test0.2016stablepoint LIMIT 1,25000")
cursor.execute(query)
result=cursor.fetchall()
y_list = np.array(result)
#print y_list
#print np.hstack((train_x,y_list))
x_list = np.hstack((train_x,y_list))
result = []
sequence_length = 5
for index in range(len(x_list) - sequence_length + 1):
    result.append(x_list[index: index + sequence_length])
result = np.array(result)
#np.random.seed(10)
#np.random.shuffle(result)
#print result
row1 = int(round(0.8 * result.shape[0]))
row2 = int(round(0.9 * result.shape[0]))
train = result[:row1, :]
test = result[row2:, :]
np.random.seed(10)
np.random.shuffle(train)
X_train = train[:,:,0:5]
y_train = train[:,-1, -1]
np.random.seed(10)
np.random.shuffle(test)
X_test = test[:,:,0:5]
y_test = test[:,-1, -1]
y_train = np.reshape(y_train, (y_train.shape[0],1))
y_test = np.reshape(y_test, (y_test.shape[0],1))
cursor.close()
cnx.close()
#print X_train
#print y_train

Using TensorFlow backend.


In [2]:
X_train4D = X_train.reshape(X_train.shape[0],5,5,1).astype('float32')
X_test4D = X_test.reshape(X_test.shape[0],5,5,1).astype('float32')
print X_train4D.shape
print X_test4D.shape
print y_train.shape
print y_test.shape

(19997, 5, 5, 1)
(2500, 5, 5, 1)
(19997, 1)
(2500, 1)


In [3]:
model = Sequential()
model.add(Conv2D(filters=4,
                kernel_size=(2,2),
                padding='same',
                input_shape=(5,5,1),
                activation='relu'))

model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(200,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(200,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='linear'))
model.compile(loss='mse', optimizer='adam',metrics=['mae', 'mape'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 5, 5, 4)           20        
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 4)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
__________

In [4]:
starttime = datetime.datetime.now()
model.fit(
    X_train4D, y_train,
    batch_size=50, epochs=80,validation_split=0.1, verbose=0)
endtime = datetime.datetime.now()
print 'train time '+str((endtime - starttime).seconds)+' s'

train time 59 s


In [5]:
predicted = model.predict(X_test4D)
#print predicted

In [1]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(y_test[0:50],label="Real")
ax.legend(loc='upper left')
plt.plot(predicted[0:50],label="Prediction")
plt.legend(loc='upper left')
plt.ylim(10,60)
acc = np.ones((y_test.shape[0],1))-abs((predicted-y_test)/y_test)
acc = round(np.mean(acc),5)
plt.title('Accuracy:'+str(100*acc)+'%')
plt.show()
from keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

NameError: name 'y_test' is not defined

In [6]:
print y_test[21:40]

[[49.05828857]
 [46.82295227]
 [47.70108032]
 [47.6730957 ]
 [46.76554871]
 [46.97032166]
 [48.93244171]
 [45.78124237]
 [49.02342224]
 [48.89259338]
 [49.0329361 ]
 [48.78486633]
 [46.88672638]
 [46.80773926]
 [49.05844116]
 [49.04650879]
 [46.90593719]
 [48.6788559 ]
 [46.81910706]]


In [7]:
a = np.load('/home/wuhang/LSTM_code/testout.npy')
print a[21:40]

[[49.0446701 ]
 [46.8411026 ]
 [47.73368835]
 [47.68197632]
 [46.79357147]
 [46.96838379]
 [48.8134613 ]
 [45.82444763]
 [49.02522278]
 [49.03032684]
 [49.0329361 ]
 [48.76895142]
 [46.88600922]
 [46.82792664]
 [49.04090881]
 [48.63806915]
 [46.89931488]
 [49.28115082]
 [46.79467773]]


In [3]:
import numpy as np
import mysql.connector
from sklearn import preprocessing
cnx = mysql.connector.connect(user='root', password='941012',
                              host='127.0.0.1',
                              database='test0')
cursor = cnx.cursor()
query = ("SELECT n2,P1,T1,P2,T2 FROM test0.2016stablepoint LIMIT 1,25000")
cursor.execute(query)
result=cursor.fetchall()
x_list = np.array(result)
scaler = preprocessing.StandardScaler().fit(x_list)
train_x = scaler.transform(x_list)
#print train_x
query = ("SELECT P34 FROM test0.2016stablepoint LIMIT 1,25000")
cursor.execute(query)
result=cursor.fetchall()
y_list = np.array(result)
#print y_list
#print np.hstack((train_x,y_list))
x_list = np.hstack((train_x,y_list))
result = []
sequence_length = 5
for index in range(len(x_list) - sequence_length + 1):
    result.append(x_list[index: index + sequence_length])
result = np.array(result)
#np.random.seed(10)
#np.random.shuffle(result)
#print result
row1 = int(round(0.8 * result.shape[0]))
row2 = int(round(0.9 * result.shape[0]))
train = result[:row1, :]
test = result[row2:, :]
np.random.seed(10)
np.random.shuffle(train)
X_train = train[:,:,0:5]
y_train = train[:,-1, -1]
np.random.seed(10)
np.random.shuffle(test)
X_test = test[:,:,0:5]
y_test = test[:,-1, -1]
y_train = np.reshape(y_train, (y_train.shape[0],1))
y_test = np.reshape(y_test, (y_test.shape[0],1))
cursor.close()
cnx.close()
#print X_train
#print y_train
X_train4D = X_train.reshape(X_train.shape[0],5,5,1).astype('float32')
X_test4D = X_test.reshape(X_test.shape[0],5,5,1).astype('float32')
print X_train4D.shape
a = np.array_split(X_train4D, 2, axis = 0)[1]
print a.shape

(19997, 5, 5, 1)
(9998, 5, 5, 1)
